# New generation datamodels and DBMSS Project
2023 / april 2025 edition

Before diving into reading this notebook, make sure you have read the project guidelines provided by the professor. You can find them [here](./Project2023-vers1.pdf).


## 1) Transaction Data Simulator Tool

The section focuses on how the various provided scripts were combined to produce a single versatile script that, through the use of parameters, is capable of generating CSV files containing all the data that will be inserted into the database. We will omit explaining the functionality of the Python scripts or the meaning of the data generated by the tool, as these aspects are clearly detailed on the [linked page](https://fraud-detection-handbook.github.io/fraud-detection-handbook/Chapter_3_GettingStarted/SimulatedDataset.html).

To proceed, the following Python packages and Python sources (from this project's repository) are required:

In [18]:
import os
import sys
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from neo4j import PreviewWarning
warnings.filterwarnings("ignore", category=PreviewWarning)
sys.path.append(os.path.join(os.getcwd(), '../GenerationScript/Transaction_data_simulator_code'))
from add_frauds import add_frauds
from generate_dataset import generate_dataset

### 1.1) Parameters

To manage parameters for the script in a simple way, I decided to use an array of objects. Each object represents the entire configuration for generating a single database, allowing the script to generate multiple databases with different characteristics and data volumes in one execution.

Each object in the array, so each database configuration contains:

- DB_name: The name of the database.
- n_customers: The number of customers to generate.
- n_terminals: The number of terminals to generate.
- start_date: The starting date for generating transaction dates.
- n_days: The number of days after the start_date to use for creating transaction dates.
- radius: The action radius for customers. A customer can only perform transactions at a terminal within their radius.

Here is an example:

In [19]:
DBs = [
   {
       "DB_name": "Small-DB-1",
       "n_customers": 500,
       "n_terminals": 300,
       "n_days": 7,
       "start_date": '2024-12-30',
       "radius": 10
    },
    {
        "DB_name": "Small-DB-2",
        "n_customers": 1000,
        "n_terminals": 500,
        "n_days": 14,
        "start_date": '2024-01-01',
        "radius": 5
    }
]

### 1.2) Generation Script

Below is the commented code for generating the databases using the parameters defined above.

In [20]:
output_dir = ""
# Loop sui DB definiti nel file di configurazione
for db in DBs:
    # Generazione delle tabelle del DB usando i valori di configurazione
    (customer_profiles_table, terminal_profiles_table, transactions_df) = generate_dataset(
        n_customers=db["n_customers"], 
        n_terminals=db["n_terminals"], 
        nb_days=db["n_days"], 
        start_date=db["start_date"], 
        r=db["radius"]
    )

    # Aggiungere frodi alle transazioni
    transactions_df = add_frauds(customer_profiles_table, terminal_profiles_table, transactions_df)

    
    # Converto i valori della serie available_terminals dato che gli interi nella lista sono interi numpy
    customer_profiles_table['available_terminals'] = customer_profiles_table['available_terminals'].apply(
        lambda lst: [int(i) if isinstance(i, np.integer) else i for i in lst] if isinstance(lst, (list, np.array)) else lst
    )

    # Preparazione al salvataggio del DB
    output_dir = os.path.join(os.getcwd(), '..', 'Generated_DBs', db["DB_name"])


    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Salvataggio dei customers
    customer_profiles_table.to_csv(output_dir + '/customers.csv', sep=';', encoding='utf-8', index=False)

    # Salvataggio dei terminals
    terminal_profiles_table.to_csv(output_dir + '/terminals.csv', sep=';', encoding='utf-8', index=False)

    # Salvataggio delle transactions
    transactions_df.to_csv(output_dir + '/transactions.csv', sep=';', encoding='utf-8', index=False)

    print(f"Database data saved in: {os.path.abspath(output_dir)}/\n")


print("DONE! All DBs have been created")


Time to generate customer profiles table: 0.01s
Time to generate terminal profiles table: 0.00s
Time to associate terminals to customers: 0.05s
Time to generate transactions: 0.42s
Number of frauds from scenario 1: 1
Number of frauds from scenario 2: 127
Number of frauds from scenario 3: 46
Database data saved in: /mnt/1364D0FF74AFABFF/unimi/new generation/progetto/NewGenerationDBMSSProject/Generated_DBs/Small-DB-1/

Time to generate customer profiles table: 0.01s
Time to generate terminal profiles table: 0.00s
Time to associate terminals to customers: 0.09s
Time to generate transactions: 1.00s
Number of frauds from scenario 1: 16
Number of frauds from scenario 2: 852
Number of frauds from scenario 3: 210
Database data saved in: /mnt/1364D0FF74AFABFF/unimi/new generation/progetto/NewGenerationDBMSSProject/Generated_DBs/Small-DB-2/

DONE! All DBs have been created


### 1.3) CSV Generati

Now, let's take a look at the generated CSV files.

#### Customers

In [21]:
import pandas as pd
pd.read_csv(os.path.join(output_dir, 'customers.csv'), sep=';', encoding='utf-8', index_col=0)

,x_customer_id,y_customer_id,mean_amount,std_amount,mean_nb_tx_per_day,available_terminals
CUSTOMER_ID,,,,,,
0,54.881350,71.518937,62.262521,31.131260,2.179533,"[29, 87, 144, 241, 330]"
1,42.365480,64.589411,46.570785,23.285393,3.567092,"[5, 160, 242, 378, 431, 475]"
2,96.366276,38.344152,80.213879,40.106939,2.115580,"[316, 406, 447]"
3,56.804456,92.559664,11.748426,5.874213,0.348517,"[65, 94, 113, 364, 401, 433, 485]"
4,2.021840,83.261985,78.924891,39.462446,3.480049,[372]
...,...,...,...,...,...,...
995,43.216661,36.225882,58.023111,29.011555,0.305376,"[264, 309, 395, 412, 483]"
996,4.034602,51.110309,7.707631,3.853816,0.238208,"[32, 83, 194, 239, 280]"
997,37.634146,6.177907,41.619615,20.809807,0.967002,"[147, 148, 185, 413]"


#### Terminals

In [22]:
pd.read_csv(os.path.join(output_dir, 'terminals.csv'), sep=';', encoding='utf-8', index_col=0)

,x_terminal_id,y_terminal_id
TERMINAL_ID,,
0,41.702200,72.032449
1,0.011437,30.233257
2,14.675589,9.233859
3,18.626021,34.556073
4,39.676747,53.881673
...,...,...
495,81.250730,28.380183
496,52.784680,33.941672
497,55.466731,97.440347


#### Transactions

In [23]:
pd.read_csv(os.path.join(output_dir, 'transactions.csv'), sep=';', encoding='utf-8', index_col=0)

,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO
TRANSACTION_ID,,,,,,,,
0,2024-01-01 00:00:31,596,110,57.16,31,0,0,0
1,2024-01-01 00:07:56,2,316,146.00,476,0,0,0
2,2024-01-01 00:10:34,927,415,50.99,634,0,0,0
3,2024-01-01 00:10:45,568,400,44.71,645,0,0,0
4,2024-01-01 00:13:44,541,171,59.07,824,0,0,0
...,...,...,...,...,...,...,...,...
25206,2024-01-14 23:45:57,786,104,40.75,1208757,13,0,0
25207,2024-01-14 23:46:06,392,134,7.15,1208766,13,0,0
25208,2024-01-14 23:46:40,888,141,37.55,1208800,13,0,0


### 1.4) CSV Generati
In the project guidelines, it is requested to generate three databases: one with a size of 50 MB, one with 100 MB, and one with 200 MB. The database generation script does not allow you to directly define the size of the database you want to generate. Instead, you must specify all the previously viewed parameters. After conducting some tests, I defined the parameters reported in the example to generate the three databases with the desired sizes.

Note that the generated databases represent scenarios with a high volume of transactions and a limited number of customers and terminals. This characteristic should be considered when evaluating performance, as it represents the worst-case scenario that could occur in our workload.

## 2) Conceptual Model

To create the following conceptual model, I analyzed the CSV files generated by the *Transaction Data Simulator* tool. This analysis allowed me to understand the data's semantics and design a clear and simple structure that illustrates the relationships between the data to be stored in the database.

<img src="./assets/Conceptual model UML.svg" alt="UML Diagram" style="width:800px;">

### 2.2) Costraints
#### Terminal
- 0 <= `coords.x` <= 100
- 0 <= `coords.y` <= 100

#### Customer
- 0 <= `coords.x` <= 100
- 0 <= `coords.y` <= 100
- `spending_mean` >= 0
- `spending_std` >= 0
- `transactions_per_day_mean` >= 0

#### Transactions
- `amount` > 0
- 0 <= `fraud_scenario` <= 3
- 0 <= `security_feeling` <= 5



## 3) Logical Model

Before proceeding with the logical model, it is important to indicate which database I have chosen to manage the data and the decisions I made regarding the representation of the data to meet the workload requirements.

### 3.1) Database
As a database, I chose to use Neo4j due to the nature of the data, which suggests a graph structure. Infact, all the relationships present are of the N:N type, and such relationships are excellently handled by graph databases. 

Furthermore, this choice was confirmed by the workload, especially by query 3c, which involves continuous traversal of relationships up to a certain `K` value that determines when to stop. Performing this query would be extremely costly if we had to perform a join (or lookup) for each traversed relationship. 

Additionally, as we will see later, Cypher, Neo4j's query language, offers a library called APOC that will allow us to execute query 3c with impressive performance.

### 3.2) Data representation (Workload friendly)
Since Neo4j does not allow the definition of custom types or the insertion of objects within node properties, I decided to eliminate all custom types and implement them using primitive types. For the custom types representing objects, I created a property for each attribute with its corresponding primitive type. For enums, I used simple strings.

The attribute names in the logical model differ from those in the conceptual model because they are based on those used by the *Transaction Data Simulator* tool. The meaning of any unclear or newly introduced fields can be determined by:  
- Referring to the *Transaction Data Simulator* tool documentation for fields generated by the tool.  
- Reading the following paragraph, where I explain the new fields I added.  
- Consulting the project guidelines, which detail and justify the fields explicitly required in the extended database.  

As we will see later, to improve the efficiency of the workload through indexing, I decided to split the `transactions.registration` field into its components: day, month, year, and time. These components are now represented as `tx_date_day`, `tx_date_month`, `tx_date_year`, and `tx_date_time`, respectively. This division was made because many queries in the workload filter data using only the month and year of the `transactions.registration` field. If I had created an index on the entire field, it would not have been used, as the filters in the queries would only utilize a subset of the entire field. Therefore, the division was made, and a composite index was created only on the year and month fields.

The data types specified are those present in Neo4j.

<img src="./assets/Logical model UML.svg" alt="UML Diagram" style="width:800px;">

### 3.3) Costraints
#### Terminal
- 0 <= `x_terminal_id` <= 100
- 0 <= `y_terminal_id` <= 100

#### Customer
- 0 <= `x_customer_id` <= 100
- 0 <= `y_customer_id` <= 100
- `mean_amount` >= 0
- `std_amount` >= 0
- `mean_nb_tx_per_day` >= 0

#### Transactions
- `tx_amount` > 0
- 0 <= `tx_fraud_scenario` <= 3
- 0 <= `tx_security_feeling` <= 5
- `tx_date_day`, `tx_date_month`, `tx_date_year` form a correct date type object 
- `tx_date_time` forms a correct localTime object
- `tx_day_period` is one of the following strings ["morning", "afternoon", "evening", "night"]
- `tx_products_type` is one of the following strings ["high-tech", "food", "clothing", "consumable", "other"]

### 3.4) Assumptions
Since the constraints that can be implemented in Neo4j focus only on the structure and data type, and do not allow constraints on the actual values or the direction of relationships, I assume that whichever software provides the data to be inserted into the database has correctly implemented all the constraints listed above (except for the constraints on the properties `tx_date_...`, since those can be validated at the database level). In our case, we assume that the values produced by the *Transaction Data Simulator* tool are correct and comply with the constraints. 

Since Neo4j constraints also do not allow us to define the direction of relationships, it is our responsibility to ensure that, in the queries used to create relationships, we do not make mistakes and avoid generating relationships in the wrong direction.

For more detailed information, I refer you to the Neo4j [documentation](https://neo4j.com/docs/cypher-manual/current/constraints/managing-constraints/).

## 4) Neo4j Data Loading
To proceed with this section, the following Python packages are required:

In [24]:
import time 
import neo4j

To facilitate interactions with Neo4j, we will define some "kernel" functions that will be used to interface with the database. These functions will simplify managing data with Neo4j, providing reusable methods for the rest of the project.

Before defining the kernel functions, we set some configuration parameters that will be useful not only for the kernel functions themselves but also for the various queries that will be executed later in the project through the kernel functions.
Among the configuration parameters, we have:
- `customers_csv_link`, `terminals_csv_link`, `transactions_csv_link`: these are references to the previously generated CSV files. They can be local paths or network links, and we will explain in the appropriate section why we use network links. The files used in to the example **the total size of the three files is 50 MB**. However, in the performance analysis section, we will also include load times for both larger and smaller databases.
- `lines_per_commit`: useful for batch operations sent to the database through specific Cypher directives (in our case, we will use APOC). This parameter indicates how many modified or added rows should be processed before committing the data.
- `parallel_loading`: useful for the batch operations mentioned in the previous point. This parameter indicates whether the database should perform the batch operations in parallel or sequentially.

In [25]:
#config parameters
config = {
    "customers_csv_link": "https://www.dropbox.com/scl/fi/8ctho6t1xd2hn00ht36l3/customers.csv?rlkey=rgxpnpwrfesfui6mtdz1nnedd&st=dlfsg22x&dl=1",
    "terminals_csv_link": "https://www.dropbox.com/scl/fi/3p86oc2gnoo24q4qg7czc/terminals.csv?rlkey=z13kfwu7f3uezp1pu26k7qmtk&st=pt24zewq&dl=1",
    "transactions_csv_link": "https://www.dropbox.com/scl/fi/6tygyhhen8nxfqpzlg66s/transactions.csv?rlkey=q342g524lh558p9t2c39bx1gf&st=kwgctxrj&dl=1",
    "lines_per_commit": 1000,
    "parallel_loading": "true"
}

# provide neo4j db connection
def get_neo4j_connection():
    try:
        #Using environment variables (recommended): This method securely stores credentials outside the code by using environment variables.
        #uri = os.getenv('NEO4J_URI')
        #user = os.getenv('NEO4J_USERNAME')
        #password = os.getenv('NEO4J_PASSWORD')
        
        #Using plain strings (not recommended): This method directly includes credentials in the code, which exposes them to potential security risks.
        #In this case, to keep things as simple as possible, I will use plain text credentials since they are for a free version of Neo4j.
        #You can create it by following this link: https://neo4j.com/product/auradb
        uri = "neo4j+s://45d4bc57.databases.neo4j.io"
        user = "neo4j"
        password = "o8mbh0hFGILahScLJw2yTYWIwQ6z7lPhQT6m-U2W1c8"
        
        return neo4j.GraphDatabase.driver(uri, auth=(user, password))
    
    except Exception as e:
        print(f"ERROR: An unexpected error occurred while connecting to Neo4j: {e}")
        return None

# close neo4j db connection
def close_neo4j_connection(driver):
    if driver is not None:
        driver.close()

# clear the db from data, relations and costraints
def clear_database():
    driver = get_neo4j_connection()
    if driver is None:
        return False

    delete_nodes_query = f"""
        MATCH (n)
        CALL apoc.nodes.delete(n, {config["lines_per_commit"]}) YIELD value
        RETURN value
    """
    
    try:
        start_time=time.time()
        driver.execute_query(delete_nodes_query)

        constraints_result = driver.execute_query("SHOW CONSTRAINTS").records
        for record in constraints_result:
            drop_constraint_query = "DROP CONSTRAINT $name"
            driver.execute_query(drop_constraint_query, {"name": record["name"]})

        indexes_result = driver.execute_query("SHOW INDEXES").records
        for record in indexes_result:
            drop_index_query = "DROP INDEX $name"
            driver.execute_query(drop_index_query, {"name": record["name"]})

        print("clear_database execution time: {:.2f}s".format(time.time()-start_time))
        return True

    except Exception as e:
        print(f"ERROR clear_database: {e}")
        return False

    finally:
        close_neo4j_connection(driver)

# Performs a query that does not expect data as a result
def execute_query_command(name, query):
    driver = get_neo4j_connection()
    try:
        start_time=time.time()
        driver.execute_query(query)
        print(f"{name} execution time: {{:.2f}}s".format(time.time()-start_time))
        return True
    except Exception as e:
        print(f"ERROR {name}: {e}")
        return False
    finally:
        close_neo4j_connection(driver)
        
# Performs some querys where each one does not expect data as a result
def execute_query_commands(name, queries):
    driver = get_neo4j_connection()
    try:
        start_time=time.time()
        
        for query in queries:
            driver.execute_query(query)

        print(f"{name} execution time: {{:.2f}}s".format(time.time()-start_time))
        return True
    except Exception as e:
        print(f"ERROR {name}: {e}")
        return False
    finally:
        close_neo4j_connection(driver)

# performs a query that returns data and converts it to a dataframe
def execute_query_df(name, query):
    driver = get_neo4j_connection()
    if driver is None:
        return False

    try:
        start_time=time.time()
        result = driver.execute_query(query, result_transformer_= neo4j.Result.to_df)
        print(f"{name} execution time: {{:.2f}}s".format(time.time() - start_time))

        return result
    except Exception as e:
        print(f"ERROR {name}: {e}")
        return None
    finally:
        close_neo4j_connection(driver)

**Let’s begin by cleaning the database.** This step is unnecessary if you have just created a new database instance, but if you are reusing an instance on which you have already performed some operations, such as running this notebook before, it is advisable to restore it to its original state by clearing everything. In this case, the `clear_database()` function comes to our aid.

In [26]:
clear_database()

clear_database execution time: 4.42s


True

### 4.1) Schema
Neo4j constraints focus solely on the data structure, as they are used to define a schema for the data. Thanks to Neo4j's schemaless nature, or more generally the schemaless nature of NoSQL databases, it is possible to insert data with maximum flexibility, without the need to define a formal schema in advance. This flexibility allows for handling heterogeneous data and adapting to changes over time, making it ideal for scenarios where the data structure may evolve.

However, despite this flexibility, defining a schema is still considered good practice. It provides several benefits, particularly in terms of performance when running queries that filter data or when calculations need to be performed on the data. By enforcing data types and data presence through the schema, the database can optimize certain operations, especially those that involve processing already present values. On the other hand, one drawback of using a schema is that it requires additional processing during insertions and modifications, as the database must validate that each new piece of data complies with the defined constraints.

The schema we are about to define in the database involves taking the previously documented logical model and:
- adding constraints that associate each attribute with its respective type;
- defining, for each entity (from the logical model), the attributes that form the primary key.
- Adding constraints that make the attributes mandatory, for attributes not specified as primary keys, since they are already mandatory due to the primary key constraint.

In [27]:
def create_terminals_schema():
    queries = [
        "CREATE CONSTRAINT terminal_id_is_integer FOR (t:Terminal) REQUIRE t.terminal_id IS :: INTEGER;",
        "CREATE CONSTRAINT terminal_id_key FOR (t:Terminal) REQUIRE t.terminal_id IS NODE KEY;",
        "CREATE CONSTRAINT terminal_x_is_float FOR (t:Terminal) REQUIRE t.x_terminal_id IS :: FLOAT;",
        "CREATE CONSTRAINT terminal_x_required FOR (t:Terminal) REQUIRE t.x_terminal_id IS NOT NULL;",
        "CREATE CONSTRAINT terminal_y_is_float FOR (t:Terminal) REQUIRE t.y_terminal_id IS :: FLOAT;",
        "CREATE CONSTRAINT terminal_y_required FOR (t:Terminal) REQUIRE t.y_terminal_id IS NOT NULL;"
    ]
    
    return execute_query_commands("create_terminals_schema", queries)

def create_customers_schema():
    queries = [
        "CREATE CONSTRAINT customer_id_is_integer FOR (c:Customer) REQUIRE c.customer_id IS :: INTEGER;",
        "CREATE CONSTRAINT customer_id_key FOR (c:Customer) REQUIRE c.customer_id IS NODE KEY;",
        "CREATE CONSTRAINT customer_x_is_float FOR (c:Customer) REQUIRE c.x_customer_id IS :: FLOAT;",
        "CREATE CONSTRAINT customer_x_required FOR (c:Customer) REQUIRE c.x_customer_id IS NOT NULL;",
        "CREATE CONSTRAINT customer_y_is_float FOR (c:Customer) REQUIRE c.y_customer_id IS :: FLOAT;",
        "CREATE CONSTRAINT customer_y_required FOR (c:Customer) REQUIRE c.y_customer_id IS NOT NULL;",
        "CREATE CONSTRAINT customer_mean_amount_is_float FOR (c:Customer) REQUIRE c.mean_amount IS :: FLOAT;",
        "CREATE CONSTRAINT customer_mean_amount_required FOR (c:Customer) REQUIRE c.mean_amount IS NOT NULL;",
        "CREATE CONSTRAINT customer_std_amount_is_float FOR (c:Customer) REQUIRE c.std_amount IS :: FLOAT;",
        "CREATE CONSTRAINT customer_std_amount_required FOR (c:Customer) REQUIRE c.std_amount IS NOT NULL;",
        "CREATE CONSTRAINT customer_mean_nb_tx_per_day_is_float FOR (c:Customer) REQUIRE c.mean_nb_tx_per_day IS :: FLOAT;",
        "CREATE CONSTRAINT customer_mean_nb_tx_per_day_required FOR (c:Customer) REQUIRE c.mean_nb_tx_per_day IS NOT NULL;"
    ]
    return execute_query_commands("create_customers_schema", queries)

def create_transaction_schema():
    queries = [
        "CREATE CONSTRAINT transaction_id_is_integer FOR ()-[transaction:Make_transaction]->() REQUIRE transaction.transaction_id IS :: INTEGER;",
        "CREATE CONSTRAINT transaction_id_key FOR ()-[transaction:Make_transaction]->() REQUIRE transaction.transaction_id IS RELATIONSHIP KEY;",
        "CREATE CONSTRAINT tx_time_seconds_is_integer FOR ()-[transaction:Make_transaction]->() REQUIRE transaction.tx_time_seconds IS :: INTEGER;",
        "CREATE CONSTRAINT tx_time_seconds_required FOR ()-[transaction:Make_transaction]->() REQUIRE transaction.tx_time_seconds IS NOT NULL;",
        "CREATE CONSTRAINT tx_time_days_is_integer FOR ()-[transaction:Make_transaction]->() REQUIRE transaction.tx_time_days IS :: INTEGER;",
        "CREATE CONSTRAINT tx_time_days_required FOR ()-[transaction:Make_transaction]->() REQUIRE transaction.tx_time_days IS NOT NULL;",
        "CREATE CONSTRAINT tx_amount_is_float FOR ()-[transaction:Make_transaction]->() REQUIRE transaction.tx_amount IS :: FLOAT;",
        "CREATE CONSTRAINT tx_amount_required FOR ()-[transaction:Make_transaction]->() REQUIRE transaction.tx_amount IS NOT NULL;",
        "CREATE CONSTRAINT tx_date_day_required FOR ()-[transaction:Make_transaction]->() REQUIRE transaction.tx_date_day IS NOT NULL;",
        "CREATE CONSTRAINT tx_date_day_is_integer FOR ()-[transaction:Make_transaction]->() REQUIRE transaction.tx_date_day IS :: INTEGER;",
        "CREATE CONSTRAINT tx_date_month_is_integer FOR ()-[transaction:Make_transaction]->() REQUIRE transaction.tx_date_month IS :: INTEGER;",
        "CREATE CONSTRAINT tx_date_month_required FOR ()-[transaction:Make_transaction]->() REQUIRE transaction.tx_date_month IS NOT NULL;",
        "CREATE CONSTRAINT tx_date_year_is_integer FOR ()-[transaction:Make_transaction]->() REQUIRE transaction.tx_date_year IS :: INTEGER;",
        "CREATE CONSTRAINT tx_date_year_required FOR ()-[transaction:Make_transaction]->() REQUIRE transaction.tx_date_year IS NOT NULL;",
        "CREATE CONSTRAINT tx_date_time_is_localtime FOR ()-[transaction:Make_transaction]->() REQUIRE transaction.tx_date_time IS :: LOCAL TIME;",
        "CREATE CONSTRAINT tx_date_time_required FOR ()-[transaction:Make_transaction]->() REQUIRE transaction.tx_date_time IS NOT NULL;",
        "CREATE CONSTRAINT tx_fraud_is_boolean FOR ()-[transaction:Make_transaction]->() REQUIRE transaction.tx_fraud IS :: BOOLEAN;",
        "CREATE CONSTRAINT tx_fraud_is_required FOR ()-[transaction:Make_transaction]->() REQUIRE transaction.tx_fraud IS NOT NULL;",
        "CREATE CONSTRAINT tx_fraud_scenario_is_integer FOR ()-[transaction:Make_transaction]->() REQUIRE transaction.tx_fraud_scenario IS :: INTEGER;",
        "CREATE CONSTRAINT tx_fraud_scenario_is_required FOR ()-[transaction:Make_transaction]->() REQUIRE transaction.tx_fraud_scenario IS NOT NULL;"
    ]
    return execute_query_commands("create_transaction_schema", queries)

create_terminals_schema()
create_customers_schema()
create_transaction_schema()

create_terminals_schema execution time: 1.08s
create_customers_schema execution time: 1.78s
create_transaction_schema execution time: 2.85s


True

### 4.2) Data loading

To load data into Neo4j using CSV files, we must first consider where the Neo4j instance resides in which we want to load the data. This aspect is crucial because the CSV files must be accessible from the machine running the Neo4j instance. This results in two possible scenarios:
- The CSV files reside on the machine where the Neo4j instance is running,
- The CSV files are network resources that can be directly downloaded via a link.

Since we are using a Neo4j instance managed by an external company, Aura, they obviously do not provide us access to their servers, so we must opt for the second option.

This will have an impact on the data loading performance, as the time indicated by the loading procedure will not only account for the time required to load the data from the file to the database but will also include the time for the Neo4j instance to download the file. The download time is not negligible because, as we know, the network is much slower compared to a completely local approach. Check it yourself by pasting the transactions CSV file URL into your browser and seeing how long it takes for your machine to download the file.

It’s important to use a direct download link for the CSV files to ensure everything works. For easily and quickly sharing these files, I chose Dropbox because it offers a file sharing option with links that include a query parameter in the URL. This parameter, appearing as `&dl=1` at the end of the link, allows me to specify whether the link should be a direct download. This feature is crucial for the Neo4j instance to download the file correctly. I also explored other cloud storage systems, but the process of obtaining a direct download link was unnecessarily more complex.

Now let's look at the queries used to load the data into the database. Initially, I wanted to load the data using the same example provided by the professor during the lessons, where a Cypher directive was used to load data from a CSV file in batches of N rows per commit. However, since this directive has been deprecated, I opted to use the APOC library, which allowed me to achieve the same behavior.

In [28]:
def load_terminals_from_csv():
    query = f"""
        CALL apoc.periodic.iterate(
            'LOAD CSV WITH HEADERS FROM "{config["terminals_csv_link"]}" AS row FIELDTERMINATOR ";" 
            RETURN row',
            'MERGE (t:Terminal {{terminal_id: toInteger(row.TERMINAL_ID)}})
            ON CREATE SET 
                t.x_terminal_id = toFloat(row.x_terminal_id),
                t.y_terminal_id = toFloat(row.y_terminal_id)
            ',
            {{batchSize: {config["lines_per_commit"]}, parallel: {config["parallel_loading"]}}}
        )
    """
    return execute_query_command("load_terminals_from_csv", query)

def load_customers_with_available_terminals_from_csv():    
    query = f"""
        CALL apoc.periodic.iterate(
            'LOAD CSV WITH HEADERS FROM "{config["customers_csv_link"]}" AS row FIELDTERMINATOR ";" 
            RETURN row',
            'MERGE (c:Customer {{customer_id: toInteger(row.CUSTOMER_ID)}})
            ON CREATE SET  
                c.x_customer_id = toFloat(row.x_customer_id),
                c.y_customer_id = toFloat(row.y_customer_id),
                c.mean_amount = toFloat(row.mean_amount),
                c.std_amount = toFloat(row.std_amount),
                c.mean_nb_tx_per_day = toFloat(row.mean_nb_tx_per_day)
            WITH c, row
            WITH c, apoc.convert.fromJsonList(row.available_terminals) AS available_terminal_ids
            UNWIND available_terminal_ids AS available_terminal_id
            MATCH (t:Terminal {{terminal_id: available_terminal_id}})
            MERGE (c)-[:Available]->(t)
            ',
            {{batchSize: {config["lines_per_commit"]}, parallel: {config["parallel_loading"]}}}
        )
    """
    return execute_query_command("load_customers_with_available_terminals_from_csv",query)

def load_transactions_from_csv():
    query = f"""
        CALL apoc.periodic.iterate(
            'LOAD CSV WITH HEADERS FROM "{config["transactions_csv_link"]}" AS row FIELDTERMINATOR ";" 
            RETURN row',
            'WITH row,
                  split(row.TX_DATETIME, " ") AS splitted_date_time
                  
            WITH row,
                 date(splitted_date_time[0]) AS parsed_date,
                 localtime(splitted_date_time[1]) AS parsed_local_time

            MATCH (c:Customer {{customer_id: toInteger(row.CUSTOMER_ID)}}), 
                (t:Terminal {{terminal_id: toInteger(row.TERMINAL_ID)}})
            MERGE (c)-[transaction:Make_transaction {{transaction_id: toInteger(row.TRANSACTION_ID)}}]->(t)
            ON CREATE SET 
                transaction.tx_time_seconds = toInteger(row.TX_TIME_SECONDS), 
                transaction.tx_time_days = toInteger(row.TX_TIME_DAYS),
                transaction.tx_amount = toFloat(row.TX_AMOUNT), 
                transaction.tx_fraud = toBoolean(toInteger(row.TX_FRAUD)), 
                transaction.tx_fraud_scenario = toInteger(row.TX_FRAUD_SCENARIO),

                transaction.tx_date_day = parsed_date.day,
                transaction.tx_date_month = parsed_date.month,
                transaction.tx_date_year = parsed_date.year, 
                transaction.tx_date_time = parsed_local_time 
            ',
            {{batchSize: {config["lines_per_commit"]}, parallel: {config["parallel_loading"]}}}
        )
    """
    return execute_query_command("load_transactions_from_csv",query)

load_terminals_from_csv()
load_customers_with_available_terminals_from_csv()
load_transactions_from_csv()


load_terminals_from_csv execution time: 2.08s
load_customers_with_available_terminals_from_csv execution time: 3.46s
load_transactions_from_csv execution time: 77.68s


True

## 5) Workload
In this section, I’ll explain how I implemented the queries to efficiently respond to the various requests outlined in the project specifications. Since the requested queries were not always precise in every detail, each query’s analysis will follow these key points:
- Report the query as expressed in the project specifications.
- Explain my interpretation of the query.
- Explain how i have built the query, providing the query code
- Look at the results
- Evaluate the query's performance.

Others queries performance details will be included in the dedicated section, where the execution times of the various queries will be compared across databases of different sizes.

**An important note:** since I couldn’t find a way to clear the caches in the free Neo4j instance (and I don’t think it’s possible), when comparing the execution times of different versions of the same query, or the same query on different DBs, **it’s important to ensure the accuracy of the timings by running them multiple times**, so we will look at the chached queries timing.

### 5.1) Query A
#### 5.1.1) Query request
> For each customer checks that the spending frequency and the spending amounts of the last month is under the usual spending frequency and the spending amounts for the same period.

- "for each customer": this indicates that the query results must include all customers, even those for whom it is not possible to calculate the requested data.  

- "of the last month": refers to the month preceding the one provided as a parameter in the query. To call the python function that executes this query you have to specify partial date in the "yyyy-MM" format as a parameter. This date is then used to calculate the `first_of_previous_month` variable within the query. This variable represents the first day of the month immediately prior to the given date. When determining the value of `first_of_previous_month`, only the month and year are considered, ensuring that the query correctly filters data relevant to the previous month.  

- "usual spending frequency and the spending amounts for the same period": I interpreted this to mean that the spending frequency and spending amount must be calculated as the average of all spending frequencies and amounts recorded in the database that match the same month but correspond to a year earlier than the `first_of_previous_month` variable.

#### 5.1.2) A1 query code
Let's provide a first version of the query A

The query starts by calculating the date corresponding to the first day of the previous month relative to the date provided to the Python function. This date is saved in the variable `first_of_previous_month`.

Next, all customers are matched to ensure that none are excluded from the final result of the query. This is done because the following `WHERE` clauses do not filter out customers, and all subsequent matches are `OPTIONAL MATCH`.

The first `OPTIONAL MATCH` is used to retrieve the transaction history for the same period, this transactions are stored in the variable `tx_prev_month_all_prev_year`.

The subsequent `WITH` clause is particular because, instead of counting the `tx_prev_month_all_prev_year` and summing their amounts, it returns `NULL` for both values, if no transactions are found in the history. This is useful for differentiating, in the final result, customers for whom no significant transaction history is found (and therefore no calculations can be made) from those for whom a history is available (and calculations can proceed as required by the query).

The next `WITH` clause calculates the averages of the just computed results `tx_prev_month_prev_year_total_amount` and `tx_prev_month_prev_year_montly_freq` obtaining  `tx_prev_month_all_prev_year_total_amount_avg` and `tx_prev_month_all_prev_year_montly_freq_avg`. The `AVG` operator preserves the `NULL` value when calculating based on `NULL`; thus, if there are no transactions, `AVG(NULL)` will return `NULL`.

The last `OPTIONAL MATCH` performs the same calculations as the previous one, but now on transactions `tx` that have same month and year as `first_of_previous_month`. Unlike before, distinguishing between customers with and without transactions is not required at this stage, as this distinction will be handled in the `RETURN` clause by referencing the historical data.

The last `WITH` calculates `total_amount_prev_month` and `monthly_freq_prev_month`, which represent the total transactions amount and transaction frequency of all the `tx`s. These two values are then used in the `RETURN` stage to determine if they fall below the usual average transactions amounts and frequency.

In the `RETURN` statement, if the customer has historical data for the same period (indicated by `tx_prev_month_all_prev_year_monthly_freq_avg IS NOT NULL`), we proceed to check whether `total_amount_prev_month < tx_prev_month_all_prev_year_total_amount_avg` and `monthly_freq_prev_month < tx_prev_month_all_prev_year_monthly_freq_avg`. It is important to note that, in this scenario, the customer may not have any `tx`s. However, since historical data is available, the absence of `tx`s does not indicate missing data in the database. Instead, it signifies that the customer did not perform any transactions during the same month and year as `first_of_previous_month`.

If a customer hasn't the same period historical data we cannot provide any meaningful response so we repond with `NULL` value in both column `is_under_total_amount_avg_of_same_period` and `is_under_monthly_freq_avg_of_same_period`.

In [36]:
#year_and_month_under_analesis is a string that contains a year and a month in the format yyyy-MM
def query_a1(year_and_month_under_analesis):
    query = f"""
            WITH date.truncate('month', date("{year_and_month_under_analesis}" + "-01") ) - duration({{months: 1}}) AS first_of_previous_month
            
            MATCH (c:Customer)

            OPTIONAL MATCH (c)-[tx_prev_month_all_prev_year:Make_transaction]->(:Terminal)
            WHERE 
                tx_prev_month_all_prev_year.tx_date_month = first_of_previous_month.month
                AND tx_prev_month_all_prev_year.tx_date_year < first_of_previous_month.year
            WITH
                first_of_previous_month,
                c,
                tx_prev_month_all_prev_year.tx_date_year as year, 
                CASE 
                    WHEN COUNT(tx_prev_month_all_prev_year)>0 THEN SUM(tx_prev_month_all_prev_year.tx_amount)
                    ELSE NULL
                END AS tx_prev_month_prev_year_total_amount, 

                CASE 
                    WHEN  COUNT(tx_prev_month_all_prev_year)>0 THEN COUNT(tx_prev_month_all_prev_year)
                    ELSE NULL
                END AS tx_prev_month_prev_year_montly_freq
            WITH
            first_of_previous_month,
            c, 
            AVG(tx_prev_month_prev_year_total_amount) AS tx_prev_month_all_prev_year_total_amount_avg, 
            AVG(tx_prev_month_prev_year_montly_freq) AS tx_prev_month_all_prev_year_montly_freq_avg

            OPTIONAL MATCH (c)-[tx:Make_transaction]->(:Terminal)
            WHERE 
                tx.tx_date_month = first_of_previous_month.month AND 
                tx.tx_date_year = first_of_previous_month.year
            WITH
                c,
                SUM(tx.tx_amount) AS total_amount_prev_month, 
                COUNT(tx) AS monthly_freq_prev_month,
                tx_prev_month_all_prev_year_total_amount_avg,
                tx_prev_month_all_prev_year_montly_freq_avg

            RETURN
                c,

                CASE 
                    WHEN tx_prev_month_all_prev_year_total_amount_avg IS NULL THEN NULL
                    ELSE total_amount_prev_month < tx_prev_month_all_prev_year_total_amount_avg
                END AS is_under_total_amount_avg_of_same_period,

                CASE 
                    WHEN tx_prev_month_all_prev_year_montly_freq_avg IS NULL THEN NULL
                    ELSE monthly_freq_prev_month < tx_prev_month_all_prev_year_montly_freq_avg
                END AS is_under_monthly_freq_avg_of_same_period
    """

    return execute_query_df("query_a1",query)

month_and_year_under_analesis = "2023-05"
query_a1(month_and_year_under_analesis)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.AggregationSkippedNull} {category: UNRECOGNIZED} {title: The query contains an aggregation function that skips null values.} {description: null value eliminated in set function.} {position: None} for query: '\n            WITH date.truncate(\'month\', date("2023-05" + "-01") ) - duration({months: 1}) AS first_of_previous_month\n            \n            MATCH (c:Customer)\n\n            OPTIONAL MATCH (c)-[tx_prev_month_all_prev_year:Make_transaction]->(:Terminal)\n            WHERE \n                tx_prev_month_all_prev_year.tx_date_month = first_of_previous_month.month\n                AND tx_prev_month_all_prev_year.tx_date_year < first_of_previous_month.year\n            WITH\n                first_of_previous_month,\n                c,\n                tx_prev_month_all_prev_year.tx_date_year as year, \n                CASE \n                    WHEN COUNT(tx_prev_month_all_prev_y

query_a1 execution time: 4.22s


,c,is_under_total_amount_avg_of_same_period,is_under_monthly_freq_avg_of_same_period
0,"(mean_amount, x_customer_id, mean_nb_tx_per_da...",True,True
1,"(mean_amount, x_customer_id, mean_nb_tx_per_da...",True,True
2,"(mean_amount, x_customer_id, mean_nb_tx_per_da...",True,True
3,"(mean_amount, x_customer_id, mean_nb_tx_per_da...",None,None
4,"(mean_amount, x_customer_id, mean_nb_tx_per_da...",False,False
...,...,...,...
995,"(mean_amount, x_customer_id, mean_nb_tx_per_da...",False,True
996,"(mean_amount, x_customer_id, mean_nb_tx_per_da...",True,True
997,"(mean_amount, x_customer_id, mean_nb_tx_per_da...",False,False
998,"(mean_amount, x_customer_id, mean_nb_tx_per_da...",True,False


#### 5.1.3) A1 Performances

To improve the query performance since it fiters the data on `make_transaction.tx_date_month` and `make_transaction.tx_date_year` we can build a compound index on these two fiels
after that we can call again the query passing the same argument and look at the execution time

In [30]:
def create_composite_index_if_not_exists_on_Make_transaction_tx_date_month_and_tx_date_year():
    query = "CREATE INDEX composite_index_on_tx_date_year_and_month IF NOT EXISTS FOR ()-[tx:Make_transaction]-() ON (tx.tx_date_month, tx.tx_date_year)"
    return execute_query_command("create_composite_index_if_not_exists_on_Make_transaction_tx_date_month_and_tx_date_year", query)

create_composite_index_if_not_exists_on_Make_transaction_tx_date_month_and_tx_date_year()

create_composite_index_if_not_exists_on_Make_transaction_tx_date_month_and_tx_date_year execution time: 0.45s


True

In [31]:
query_a1(month_and_year_under_analesis)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.AggregationSkippedNull} {category: UNRECOGNIZED} {title: The query contains an aggregation function that skips null values.} {description: null value eliminated in set function.} {position: None} for query: '\n            WITH date.truncate(\'month\', date("2026-05" + "-01") ) - duration({months: 1}) AS first_of_previous_month\n            \n            MATCH (c:Customer)\n\n            OPTIONAL MATCH (c)-[tx_prev_month_all_prev_year:Make_transaction]->(:Terminal)\n            WHERE \n                tx_prev_month_all_prev_year.tx_date_month = first_of_previous_month.month\n                AND tx_prev_month_all_prev_year.tx_date_year < first_of_previous_month.year\n            WITH\n                first_of_previous_month,\n                c,\n                tx_prev_month_all_prev_year.tx_date_year as year, \n                CASE \n                    WHEN COUNT(tx_prev_month_all_prev_y

query_a1 execution time: 6.61s


,c,is_under_total_amount_avg_of_same_period,is_under_monthly_freq_avg_of_same_period
0,"(mean_amount, x_customer_id, mean_nb_tx_per_da...",True,True
1,"(mean_amount, x_customer_id, mean_nb_tx_per_da...",True,True
2,"(mean_amount, x_customer_id, mean_nb_tx_per_da...",True,True
3,"(mean_amount, x_customer_id, mean_nb_tx_per_da...",True,True
4,"(mean_amount, x_customer_id, mean_nb_tx_per_da...",True,True
...,...,...,...
995,"(mean_amount, x_customer_id, mean_nb_tx_per_da...",True,True
996,"(mean_amount, x_customer_id, mean_nb_tx_per_da...",True,True
997,"(mean_amount, x_customer_id, mean_nb_tx_per_da...",True,True
998,"(mean_amount, x_customer_id, mean_nb_tx_per_da...",True,True


As shown in the execution plan image below, the query is not utilizing the index at all! This occurs because, in the initial `MATCH` clause, we are not directly filtering the transactions. Instead, we first match the customers, which prevents the query from leveraging the index efficiently.  

In fact, the only index used is on the customers, and it is applied merely to retrieve all customer nodes without performing any filtering. Regarding transactions, no index is utilized either in the initial filtering or in the subsequent `OPTIONAL MATCH`, further contributing to the inefficiency of the query.  

To generate the execution plan shown in the image, you simply need to prefix the query with the word `EXPLAIN` in Neo4j.  

<img src="./assets/Execution plan query A1.svg" style="width:600px;">

#### 5.1.4) A2 query code
By slightly modifying the query to omit the "for all customers" clause and displaying only customers with historical data, we can significantly improve performance by leveraging the index. This optimization involves removing the initial `MATCH` clause, turning the second `OPTIONAL MATCH` into a regular `MATCH`.  

This change means that the results will no longer include customers with `NULL` values in columns `tx_prev_month_all_prev_year_total_amount_avg` and `tx_prev_month_all_prev_year_montly_freq_avg`, as these customers will be excluded directly by the first `MATCH` clause.  


In [37]:
#year_and_month_under_analesis is a string that contains a year and a month in the format yyyy-MM
def query_a2(year_and_month_under_analesis):
    query = f"""
            WITH date.truncate('month', date("{year_and_month_under_analesis}" + "-01") ) - duration({{months: 1}}) AS first_of_previous_month

            MATCH (c)-[tx_prev_month_all_prev_year:Make_transaction]->(:Terminal)
            WHERE 
                tx_prev_month_all_prev_year.tx_date_month = first_of_previous_month.month
                AND tx_prev_month_all_prev_year.tx_date_year < first_of_previous_month.year
            WITH
                first_of_previous_month,
                c,
                tx_prev_month_all_prev_year.tx_date_year as year,
                SUM(tx_prev_month_all_prev_year.tx_amount)  AS tx_prev_month_prev_year_total_amount, 
                COUNT(tx_prev_month_all_prev_year) AS tx_prev_month_prev_year_montly_freq
            WITH
            first_of_previous_month,
            c, 
            AVG(tx_prev_month_prev_year_total_amount) AS tx_prev_month_all_prev_year_total_amount_avg, 
            AVG(tx_prev_month_prev_year_montly_freq) AS tx_prev_month_all_prev_year_montly_freq_avg

            OPTIONAL MATCH (c)-[tx:Make_transaction]->(:Terminal)
            WHERE 
                tx.tx_date_month = first_of_previous_month.month AND 
                tx.tx_date_year = first_of_previous_month.year
            WITH
                c,
                SUM(tx.tx_amount) AS total_amount_prev_month, 
                COUNT(tx) AS monthly_freq_prev_month,
                tx_prev_month_all_prev_year_total_amount_avg,
                tx_prev_month_all_prev_year_montly_freq_avg

            RETURN
                c, 
                total_amount_prev_month < tx_prev_month_all_prev_year_total_amount_avg  AS is_under_total_amount_avg_of_same_period,
                monthly_freq_prev_month < tx_prev_month_all_prev_year_montly_freq_avg AS is_under_monthly_freq_avg_of_same_period
            """
    
    return execute_query_df("query_a2",query)
query_a2(month_and_year_under_analesis)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.AggregationSkippedNull} {category: UNRECOGNIZED} {title: The query contains an aggregation function that skips null values.} {description: null value eliminated in set function.} {position: None} for query: '\n            WITH date.truncate(\'month\', date("2023-05" + "-01") ) - duration({months: 1}) AS first_of_previous_month\n\n            MATCH (c)-[tx_prev_month_all_prev_year:Make_transaction]->(:Terminal)\n            WHERE \n                tx_prev_month_all_prev_year.tx_date_month = first_of_previous_month.month\n                AND tx_prev_month_all_prev_year.tx_date_year < first_of_previous_month.year\n            WITH\n                first_of_previous_month,\n                c,\n                tx_prev_month_all_prev_year.tx_date_year as year,\n                SUM(tx_prev_month_all_prev_year.tx_amount)  AS tx_prev_month_prev_year_total_amount, \n                COUNT(tx_prev_m

query_a2 execution time: 2.49s


,c,is_under_total_amount_avg_of_same_period,is_under_monthly_freq_avg_of_same_period
0,"(mean_amount, x_customer_id, mean_nb_tx_per_da...",True,True
1,"(mean_amount, x_customer_id, mean_nb_tx_per_da...",True,True
2,"(mean_amount, x_customer_id, mean_nb_tx_per_da...",True,True
3,"(mean_amount, x_customer_id, mean_nb_tx_per_da...",True,True
4,"(mean_amount, x_customer_id, mean_nb_tx_per_da...",True,False
...,...,...,...
929,"(mean_amount, x_customer_id, mean_nb_tx_per_da...",True,False
930,"(mean_amount, x_customer_id, mean_nb_tx_per_da...",False,False
931,"(mean_amount, x_customer_id, mean_nb_tx_per_da...",False,False
932,"(mean_amount, x_customer_id, mean_nb_tx_per_da...",False,False


#### 5.1.5) A2 Performances
As shown in the execution plan image below, the query is now utilizing the index we specifically created for filtering transactions. Unlike the initial version, where no index was used on the transactions, this optimized approach ensures that the query leverages the index effectively to improve performance during the filtering process.

<img src="./assets/Execution plan query A2.svg" style="width:600px;">

### 5.2) Query B
#### 5.2.1) Query request
> For each terminal identify the possible fraudulent transactions. The fraudulent transactions are those whose import is higher than 20% of the maximal import of the transactions executed on the same terminal in the last month.

- "For each terminal": This indicates that the query results must include all terminals, even those for which it is not possible to identify any fraudulent transactions.

- "In the last month": refers to data from the month preceding the one provided as a parameter. Similar to the previous query, this query is also parameterized by allowing a partial date in the "yyyy-MM" format to be passed to the python. This date is used to calculate the `first_of_previous_month` variable, which represents the first day of the month prior to the given date. Additionally, the query includes a reference to the first day of the current month, stored in the `today` variable, for further calculations or filtering as needed. 

#### 5.1.2) B1 Query Code
The query begins by saving the provided date into the today variable and computing the first day of the previous month, which is stored in `first_of_previous_month`. 

Next, all terminals are matched to ensure that none are excluded from the final result of the query. This is done because the following `WHERE` clauses do not filter out terminals, and all subsequent matches are `OPTIONAL MATCH`.

The first `OPTIONAL MATCH` retrieves transactions made on terminals during the month and year corresponding to `first_of_previous_month`. These transactions are saved in the `tx_prev_month` variable. However, some terminals may not have any transactions for the specified period, and in those cases, `tx_prev_month` will remain empty for those terminals.

Following this, the query calculates the fraud detection threshold using a `WITH` statement. The fraud amount limit, stored in the variable `tx_amount_fraud_limit`, is defined as 20% more than the maximum transaction amount from the previous month. For terminals where no transactions were found in `tx_prev_month`, the fraud amount limit remains `NULL`.

The next step uses another `OPTIONAL MATCH` to retrieve transactions for the current month, filtering by the same month and year as `today`. These transactions are stored in the `tx_current_month` variable. Using the calculated fraud amount limit, the query identifies fraudulent transactions by collecting those in `tx_current_month` where the transaction amount exceeds `tx_amount_fraud_limit`. This collection is saved in `fraud_txs_current_month`. If `tx_amount_fraud_limit` is `NULL`, the condition will always evaluate to false, resulting in an empty collection for the terminal.

Finally, the `RETURN` statement distinguishes between two problematic cases when a terminal has an empty `fraud_txs_current_month` collection. In the first case, the fraud amount limit could not be calculated, making it impossible to determine whether the terminal had fraudulent transactions. In the second case, the limit was calculated, but no fraudulent transactions were identified for that terminal in the current month. To address this ambiguity, the query replaces empty collections in `fraud_txs_current_month` with the value `NULL` whenever `tx_amount_fraud_limit IS NULL`. This approach ensures clarity in the results, differentiating between the two scenarios.

In [38]:
#year_and_month_under_analesis is a string that contains a year and a month in the format yyyy-MM
def query_b1(year_and_month_under_analesis):
    query = f"""
            WITH date("{year_and_month_under_analesis}" + "-01") AS today
            WITH today, date.truncate('month', today ) - duration({{months: 1}}) AS first_of_previous_month

            MATCH (t:Terminal)

            OPTIONAL MATCH (:Customer)-[tx_prev_month:Make_transaction]->(t)
            WHERE 
                tx_prev_month.tx_date_month = first_of_previous_month.month
                AND tx_prev_month.tx_date_year = first_of_previous_month.year

            with today, t, max(tx_prev_month.tx_amount) * 1.2 as tx_amount_fraud_limit

            OPTIONAL MATCH (:Customer)-[tx_current_month:Make_transaction]->(t)
            WHERE 
                tx_current_month.tx_date_month = today.month
                AND tx_current_month.tx_date_year = today.year

            WITH 
                t, 
                tx_amount_fraud_limit,
                COLLECT(CASE 
                    WHEN tx_current_month.tx_amount > tx_amount_fraud_limit THEN tx_current_month 
                    ELSE NULL 
                END) AS fraud_txs_current_month

            RETURN 
                t, 
                CASE 
                    WHEN tx_amount_fraud_limit IS NULL THEN NULL
                    ELSE fraud_txs_current_month
                END AS fraud_txs_current_month
            """

    return execute_query_df("query_b1",query)
query_b1(month_and_year_under_analesis)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.AggregationSkippedNull} {category: UNRECOGNIZED} {title: The query contains an aggregation function that skips null values.} {description: null value eliminated in set function.} {position: None} for query: '\n            WITH date("2023-05" + "-01") AS today\n            WITH today, date.truncate(\'month\', today ) - duration({months: 1}) AS first_of_previous_month\n\n            MATCH (t:Terminal)\n\n            OPTIONAL MATCH (:Customer)-[tx_prev_month:Make_transaction]->(t)\n            WHERE \n                tx_prev_month.tx_date_month = first_of_previous_month.month\n                AND tx_prev_month.tx_date_year = first_of_previous_month.year\n\n            with today, t, max(tx_prev_month.tx_amount) * 1.2 as tx_amount_fraud_limit\n\n            OPTIONAL MATCH (:Customer)-[tx_current_month:Make_transaction]->(t)\n            WHERE \n                tx_current_month.tx_date_month 

query_b1 execution time: 4.25s


,t,fraud_txs_current_month
0,"(y_terminal_id, terminal_id, x_terminal_id)",[]
1,"(y_terminal_id, terminal_id, x_terminal_id)",[]
2,"(y_terminal_id, terminal_id, x_terminal_id)",[]
3,"(y_terminal_id, terminal_id, x_terminal_id)",[]
4,"(y_terminal_id, terminal_id, x_terminal_id)",[]
...,...,...
495,"(y_terminal_id, terminal_id, x_terminal_id)","[(transaction_id, tx_date_year, tx_time_days, ..."
496,"(y_terminal_id, terminal_id, x_terminal_id)",[]
497,"(y_terminal_id, terminal_id, x_terminal_id)",[]
498,"(y_terminal_id, terminal_id, x_terminal_id)",[]


#### 5.1.3) B1 Performances
As we can see in the execution plan of the query shown below, the same behavior observed in the previous query occurs here as well. Specifically, the first `MATCH` clause, which matches all terminals, prevents the index from being used to filter the transactions.  

In fact, the only index used is on the terminals, and it is applied merely to retrieve all terminals nodes without performing any filtering. Regarding transactions, no index is utilized either in the initial filtering or in the subsequent `OPTIONAL MATCH`, further contributing to the inefficiency of the query.  

<img src="./assets/Execution plan query B1.svg" style="width:600px;">